In [ ]:
# Imports

%cd /content/drive/MyDrive/video_classification_rnn

import os
import numpy as np
# import random
import cv2
# from google.colab.patches import cv2_imshow
import tensorflow as tf
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten, TimeDistributed
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from models import RNN_model, GRU_model, LSTM_model, backbone_model
from load_video import load_video
from models import Models

In [ ]:
# Parameters setting

frame_width = 110
frame_height = 110

batch_size = 6
epochs = 10

dataset_path = "/content/drive/MyDrive/dataset/joon_del"

In [ ]:
# Create directories

if not os.path.exists("weights"):
    os.makedirs("weights")

### Preparing Data

In [ ]:
# Count maximum number of frames in all videos

num_classes = os.listdir(dataset_path)
count_frames = []

for class_label in num_classes:
  videos = os.listdir(os.path.join(dataset_path, class_label))

  for video in videos:
    video_path = os.path.join(dataset_path, class_label, video)
    frames = cv2.VideoCapture(video_path)

    total_frames = int(frames.get(cv2.CAP_PROP_FRAME_COUNT))
    count_frames.append(total_frames)

max_seq_len = np.max(count_frames)
num_sampels = len(count_frames)

print("numsmaples: ", num_sampels)
print("max sequence lenght: ", max_seq_len)

numsmaples:  68
max sequence lenght:  316


In [ ]:
def create_dataset():

  labels = []
  frames_dataset = []

  num_classes = os.listdir(dataset_path)
  frame_masks = np.zeros(shape=(num_sampels, max_seq_len), dtype="bool")
  video_count = 0

  for class_label in num_classes:
    videos = os.listdir(os.path.join(dataset_path, class_label))

    for video in videos:
      video_path = os.path.join(dataset_path, class_label, video)
      frames, num_frames = load_video(video_path, (frame_height, frame_width))

      padded_frames = np.zeros(shape=(max_seq_len, frame_height, frame_width, 3), dtype="float32")
      current_video_seq_len = min(max_seq_len, num_frames)

      # Normalize video frames
      for i, frame in enumerate(np.array(frames)):
          padded_frames[i, :] = frame / 255.

      frames_dataset.append(padded_frames)
      frame_masks[video_count, :current_video_seq_len] = 1

      video_count+=1

      # Create labels
      if class_label == "1" : 
        labels.append(1) 
      else: 
        labels.append(0)

  # Convert to np.array
  frames_dataset = np.array(frames_dataset)
  labels = np.array(labels)

  # Reshape labels
  labels = labels[..., np.newaxis]

  return [frames_dataset, frame_masks], labels

In [ ]:
# Call data preparing function

X, Y = create_dataset()

In [ ]:
# Splite data

X_train, X_val, mask_train, mask_val, Y_train, Y_val = train_test_split(X[0], X[1], Y, test_size = 0.2, random_state=42)

print("train data shape: ", X_train.shape, Y_train.shape, mask_train.shape, "validation data shape: ",  X_val.shape, mask_val.shape, Y_val.shape)

train data shape:  (54, 316, 110, 110, 3) (54, 1) (54, 316) validation data shape:  (14, 316, 110, 110, 3) (14, 316) (14, 1)


In [ ]:
Y_val

array([[1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0]])

### Define Models, Compile and fit

01- RNN

In [ ]:
models = Models((frame_height, frame_width), max_seq_len)
rnn_model = models.RNN_model()

rnn_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

rnn_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=batch_size, epochs=epochs)


Epoch 1/10
9/9 [==============================] - 67s 5s/step - loss: 1.0832 - accuracy: 0.3889 - val_loss: 1.1151 - val_accuracy: 0.4286
Epoch 2/10
9/9 [==============================] - 32s 4s/step - loss: 0.5457 - accuracy: 0.7407 - val_loss: 0.9189 - val_accuracy: 0.3571
Epoch 3/10
9/9 [==============================] - 33s 4s/step - loss: 0.5691 - accuracy: 0.7222 - val_loss: 1.0157 - val_accuracy: 0.5000
Epoch 4/10
9/9 [==============================] - 33s 4s/step - loss: 0.3951 - accuracy: 0.7963 - val_loss: 1.0068 - val_accuracy: 0.4286
Epoch 5/10
9/9 [==============================] - 32s 4s/step - loss: 0.3688 - accuracy: 0.8519 - val_loss: 1.0872 - val_accuracy: 0.3571
Epoch 6/10
9/9 [==============================] - 32s 4s/step - loss: 0.3594 - accuracy: 0.7963 - val_loss: 1.0553 - val_accuracy: 0.4286
Epoch 7/10
9/9 [==============================] - 32s 4s/step - loss: 0.2930 - accuracy: 0.9074 - val_loss: 1.1494 - val_accuracy: 0.3571
Epoch 8/10
9/9 [==================

In [ ]:
rnn_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/rnn_model.h5')

In [ ]:
rnn_model.summary()

#### 02- GRU

In [ ]:
models = Models((frame_height, frame_width), max_seq_len)
gru_model = models.GRU_model()

gru_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

gru_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=3, epochs=epochs)


Epoch 1/10
18/18 [==============================] - 50s 2s/step - loss: 1.1986 - accuracy: 0.3889 - val_loss: 0.8944 - val_accuracy: 0.5000
Epoch 2/10
18/18 [==============================] - 29s 2s/step - loss: 0.5820 - accuracy: 0.7037 - val_loss: 0.9712 - val_accuracy: 0.5714
Epoch 3/10
18/18 [==============================] - 29s 2s/step - loss: 0.3983 - accuracy: 0.7778 - val_loss: 1.0515 - val_accuracy: 0.5000
Epoch 4/10
18/18 [==============================] - 29s 2s/step - loss: 0.2756 - accuracy: 0.8704 - val_loss: 1.2300 - val_accuracy: 0.5000
Epoch 5/10
18/18 [==============================] - 29s 2s/step - loss: 0.2082 - accuracy: 0.9630 - val_loss: 1.2863 - val_accuracy: 0.3571
Epoch 6/10
18/18 [==============================] - 29s 2s/step - loss: 0.1608 - accuracy: 0.9630 - val_loss: 1.3291 - val_accuracy: 0.5000
Epoch 7/10
18/18 [==============================] - 29s 2s/step - loss: 0.1099 - accuracy: 0.9815 - val_loss: 1.4864 - val_accuracy: 0.3571
Epoch 8/10
18/18 [==

In [ ]:
gru_model.summary()

In [ ]:
gru_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/gru_model.h5')

#### 03- LSTM

In [ ]:
models = Models((frame_height, frame_width), max_seq_len)
lstm_model = models.LSTM_model()

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

lstm_model.fit([X_train, mask_train], Y_train, validation_data=[[X_val, mask_val], Y_val], batch_size=3, epochs=epochs)


Epoch 1/10
18/18 [==============================] - 66s 2s/step - loss: 0.9181 - accuracy: 0.4444 - val_loss: 0.8053 - val_accuracy: 0.3571
Epoch 2/10
18/18 [==============================] - 29s 2s/step - loss: 0.5576 - accuracy: 0.7222 - val_loss: 0.7721 - val_accuracy: 0.2857
Epoch 3/10
18/18 [==============================] - 30s 2s/step - loss: 0.4419 - accuracy: 0.8704 - val_loss: 1.0460 - val_accuracy: 0.5714
Epoch 4/10
18/18 [==============================] - 29s 2s/step - loss: 0.3314 - accuracy: 0.8889 - val_loss: 1.0394 - val_accuracy: 0.5000
Epoch 5/10
18/18 [==============================] - 30s 2s/step - loss: 0.2125 - accuracy: 0.9815 - val_loss: 1.0900 - val_accuracy: 0.4286
Epoch 6/10
18/18 [==============================] - 29s 2s/step - loss: 0.1803 - accuracy: 0.9815 - val_loss: 1.1997 - val_accuracy: 0.4286
Epoch 7/10
18/18 [==============================] - 29s 2s/step - loss: 0.1153 - accuracy: 1.0000 - val_loss: 1.4176 - val_accuracy: 0.5714
Epoch 8/10
18/18 [==

In [ ]:
lstm_model.summary()

In [ ]:
lstm_model.save_weights('/content/drive/MyDrive/video_classification_rnn/weights/lstm_model.h5')